# COGS 108 - Data Checkpoint

# Names

- Nhan Quach
- Joseph Morales
- Victor Wang
- Kevin Hu
- Areen Lu

<a id='research_question'></a>
# Research Question

How has social perception of profanity shifted across the past 20 years as indicated by the proportion of profane words utilized within the top 10 movies over the past 20 years and their respective age ratings, box office performance, and review scores?

# Dataset(s)

[The Movies Dataset](https://www.kaggle.com/datasets/rounakbanik/the-movies-dataset) contains metadata for 45,000 movies with 26 million ratings collected from 270,000 users for all movies stored within there. However, this dataset is limited only up till 2017. The dataset is gathered from [The Movie Database](https://www.themoviedb.org/) which is a community built move and TV database for movies from 2008 till now. We could update the original dataset to be current till 2022, however the IMDB API is limited to 40 fetches per second and [it was calculated to take roughly an entire week](https://www.themoviedb.org/talk/54f41aafc3a3683455000991) to fetch all necessary data.

[List of films that most frequently use the word F*](https://www.kaggle.com/datasets/devrimtuner/list-of-films-that-most-frequently-use-the-word) is as the title suggests. Movies on the list stretch as far back as 1983 to as recent as 2022. The only profane word studied on the dataset, however, is F*.

# Setup

In [1]:
import numpy             as np
import pandas            as pd
import seaborn           as sns
import matplotlib.pyplot as plt
import os

# Data Cleaning

Datasets gathered through the [The Movies Dataset](https://www.kaggle.com/datasets/rounakbanik/the-movies-dataset) were initially stored locally as they exceeded GitHub's limit on large file sizes. To preserve consistency and anonimity of our local repositories, we will be referring to the location of this data set as `kaggle/input`.

First, we will gather the file names of the various semi-structured data sets within our imports from Kaggle.

In [2]:
for dirname, _, filenames in os.walk('kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

kaggle/input\List of films that most frequently use the word F\archive.zip
kaggle/input\List of films that most frequently use the word F\List of films that most frequently use the word fuck11.csv
kaggle/input\The Movies Dataset\movies_metadata.csv


We are primarily interested in the meta_data associated with the movies from [The Movies Dataset](), so we will import that and perform a brief inspection of the type of data stored within the file.

In [3]:
meta_data=pd.read_csv('kaggle/input/The Movies Dataset\movies_metadata.csv', low_memory=False)

meta_data.tail()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
45461,False,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 10751, 'n...",http://www.imdb.com/title/tt6209470/,439050,tt6209470,fa,رگ خواب,Rising and falling between a man and woman.,...,NaN,0.0,90.0,"[{'iso_639_1': 'fa', 'name': 'فارسی'}]",Released,Rising and falling between a man and woman,Subdue,False,4.0,1.0
45462,False,NaN,0,"[{'id': 18, 'name': 'Drama'}]",NaN,111109,tt2028550,tl,Siglo ng Pagluluwal,An artist struggles to finish his work while a...,...,2011-11-17,0.0,360.0,"[{'iso_639_1': 'tl', 'name': ''}]",Released,NaN,Century of Birthing,False,9.0,3.0
45463,False,NaN,0,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...",NaN,67758,tt0303758,en,Betrayal,"When one of her hits goes wrong, a professiona...",...,2003-08-01,0.0,90.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,A deadly game of wits.,Betrayal,False,3.8,6.0
45464,False,NaN,0,[],NaN,227506,tt0008536,en,Satana likuyushchiy,"In a small town live two brothers, one a minis...",...,1917-10-21,0.0,87.0,[],Released,NaN,Satan Triumphant,False,0.0,0.0
45465,False,NaN,0,[],NaN,461257,tt6980792,en,Queerama,50 years after decriminalisation of homosexual...,...,2017-06-09,0.0,75.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Queerama,False,0.0,0.0


From here, we will want to list out the names of the columns as they are currently stored within the dataframe to identify columns of particular interest which would be relevant to keep.

In [4]:
column_names = list(meta_data.columns)
print(column_names)

['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id', 'imdb_id', 'original_language', 'original_title', 'overview', 'popularity', 'poster_path', 'production_companies', 'production_countries', 'release_date', 'revenue', 'runtime', 'spoken_languages', 'status', 'tagline', 'title', 'video', 'vote_average', 'vote_count']


We would not want to include movies which have not been released, so we will remove those entries from the table. For now, we will keep the `adult` column until we can find a more accurate method of pairing the age rating with the movie. 

In [5]:
# Remove movies which have not yet been released
meta_data = meta_data[meta_data['status'] == 'Released']

# Preserve relevant columns
column_names = ['adult', 'genres', 'release_date', 'revenue', 'title', 'vote_average', 'vote_count']
meta_data = meta_data[column_names]

# Drop any rows with null values according to the columns we preserved
meta_data.dropna(how='any', inplace=True)
meta_data.tail()

,adult,genres,release_date,revenue,title,vote_average,vote_count
45460,False,"[{'id': 18, 'name': 'Drama'}, {'id': 28, 'name...",1991-05-13,0.0,Robin Hood,5.7,26.0
45462,False,"[{'id': 18, 'name': 'Drama'}]",2011-11-17,0.0,Century of Birthing,9.0,3.0
45463,False,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...",2003-08-01,0.0,Betrayal,3.8,6.0
45464,False,[],1917-10-21,0.0,Satan Triumphant,0.0,0.0
45465,False,[],2017-06-09,0.0,Queerama,0.0,0.0


In [6]:
meta_data.shape

(44936, 7)

We can already see something strange with one movie titled **Grumpier Old Men** as it was somehow released, yet made 0 in revenue. Furthermore, we can see the distribution of `vote_count` vary drastically from 34 votes all the way to 5415 votes. We also do not require the release date to have as much granularity as it currently does, so let us truncate that to just the year and ensure that it is a numeric datatype.

In [7]:
# Convert the release date to only be the year
meta_data['release_date'] = pd.to_datetime(meta_data['release_date'], errors='coerce')
meta_data['release_date'] = pd.DatetimeIndex(meta_data['release_date']).year.astype('int')
meta_data.dtypes

adult            object
genres           object
release_date      int64
revenue         float64
title            object
vote_average    float64
vote_count      float64
dtype: object

In [8]:
# Drop movies which did not make any money
meta_data = meta_data[meta_data['revenue'] > 0]
meta_data.head()

,adult,genres,release_date,revenue,title,vote_average,vote_count
0,False,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",1995,373554033.0,Toy Story,7.7,5415.0
1,False,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",1995,262797249.0,Jumanji,6.9,2413.0
3,False,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",1995,81452156.0,Waiting to Exhale,6.1,34.0
4,False,"[{'id': 35, 'name': 'Comedy'}]",1995,76578911.0,Father of the Bride Part II,5.7,173.0
5,False,"[{'id': 28, 'name': 'Action'}, {'id': 80, 'nam...",1995,187436818.0,Heat,7.7,1886.0


In [9]:
meta_data.shape

(7395, 7)

In [11]:
# Drop movies which have less  25% of the votes
substantial_votes = meta_data['vote_count'].describe().describe()['25%']
meta_data         = meta_data[meta_data['vote_count'] >= substantial_votes]
meta_data.head()

,adult,genres,release_date,revenue,title,vote_average,vote_count
0,False,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",1995,373554033.0,Toy Story,7.7,5415.0
1,False,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",1995,262797249.0,Jumanji,6.9,2413.0
4,False,"[{'id': 35, 'name': 'Comedy'}]",1995,76578911.0,Father of the Bride Part II,5.7,173.0
5,False,"[{'id': 28, 'name': 'Action'}, {'id': 80, 'nam...",1995,187436818.0,Heat,7.7,1886.0
8,False,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",1995,64350171.0,Sudden Death,5.5,174.0


In [12]:
meta_data.shape

(4016, 7)